# Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import random

## scraping titulares

In [ ]:
news_url = "https://www.lavozdegalicia.es/coruna/"
response = requests.get(news_url)
soup = BeautifulSoup(response.text, 'html.parser')

news_keys = ['title', 'news_image', 'news_href']
    
list_news = [
    {
        news_keys[0]: news.find(class_="a-min-content").get_text(strip = True),
        news_keys[1]: news.find("div").find("img").get("src") if news.find("div").find("img") != None else None,
        news_keys[2]: news_url+news.find("a").get("href")
    }
    for news in soup.find_all(class_="article-min")]
    
list_news


In [ ]:
url_movies = "https://www.sensacine.com/cines/cine/E0770/"
response = requests.get(url_movies)
soup = BeautifulSoup(response.text, 'html.parser')

movie_keys = ['title','synopsis', 'image', 'link_ref']

list_movies = [
    {
        movie_keys[0]: movie.find(class_="meta-title-link").text,
        movie_keys[1]: movie.find(class_="synopsis").text.replace("\n", ""),
        movie_keys[2]: movie.find("img").get("src") if movie.find("img").get("src").startswith("https") else movie.find("img").get("data-src"),
        movie_keys[3]: "https://www.sensacine.com" + movie.find(class_="meta-title-link").get("href")
    }
    for movie in soup.find_all(class_="movie-card-theater")]

list_movies

## LEB ORO 

In [7]:
url_feb = "https://baloncestoenvivo.feb.es/resultados/ligaleboro/1/2023"
response = requests.get(url_feb)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find(id="_ctl0_MainContentPlaceHolderMaster_jornadaDataGrid")

match_list = []

leb_match_keys = ['teams', 'result', 'date', 'time']

for fila in table.find_all('tr')[1:]:
    temp_dict = {}
    
    for key, linha in zip(leb_match_keys, fila.find_all(['td', 'th'])):
        temp_dict[key] = linha.get_text(strip=True)
    
    match_list.append(temp_dict)

match_list

[{'teams': 'HESTIA MENORCA-ALIMERKA OVIEDO BALONCESTO',
  'result': '90-74',
  'date': '21/01/2024',
  'time': '12:00'},
 {'teams': 'GRUPO URETA TIZONA BURGOS-ICG FORÇA LLEIDA',
  'result': '72-79',
  'date': '20/01/2024',
  'time': '17:00'},
 {'teams': 'CACERES PATRIMONIO DE LA HUMANIDAD-GUUK GIPUZKOA BASKET',
  'result': '68-86',
  'date': '19/01/2024',
  'time': '20:45'},
 {'teams': 'AMICS CASTELLÓ-BALONCESTO FUENLABRADA',
  'result': '77-73',
  'date': '19/01/2024',
  'time': '20:30'},
 {'teams': 'RIOVERDE CLAVIJO-LONGEVIDA SAN PABLO BURGOS',
  'result': '62-79',
  'date': '21/01/2024',
  'time': '12:00'},
 {'teams': 'MOVISTAR ESTUDIANTES-REAL BETIS BALONCESTO',
  'result': '93-87',
  'date': '21/01/2024',
  'time': '12:30'},
 {'teams': 'GRUPO ALEGA CANTABRIA-LEYMA CORUÑA',
  'result': '81-83',
  'date': '20/01/2024',
  'time': '19:00'},
 {'teams': 'UEMC REAL VALLADOLID BALONCESTO-HLA ALICANTE',
  'result': '69-84',
  'date': '20/01/2024',
  'time': '19:00'},
 {'teams': 'MELILLA CI

In [8]:
ladder = soup.find(id="_ctl0_MainContentPlaceHolderMaster_clasificacionDataGrid")

ladder_list = []

leb_ladder_keys = ['position', 'team', 'matches_played', 'matches_w', 'matches_l', 'points']

for row in ladder.find_all('tr')[1:]:
    temp_dict = {}

    cells = row.find_all(['td', 'th'])
    for key, cell in zip(leb_ladder_keys, cells):
        if key == 'points':
            temp_dict[key] = cells[7].get_text(strip=True) 
        else:
            temp_dict[key] = cell.get_text(strip=True)

    ladder_list.append(temp_dict)

# Print the result
ladder_list

[{'position': '1',
  'team': 'MOVISTAR ESTUDIANTES',
  'matches_played': '18',
  'matches_w': '15',
  'matches_l': '3',
  'points': '33'},
 {'position': '2',
  'team': 'LEYMA CORUÑA',
  'matches_played': '18',
  'matches_w': '14',
  'matches_l': '4',
  'points': '32'},
 {'position': '3',
  'team': 'GUUK GIPUZKOA BASKET',
  'matches_played': '18',
  'matches_w': '13',
  'matches_l': '5',
  'points': '31'},
 {'position': '4',
  'team': 'LONGEVIDA SAN PABLO BURGOS',
  'matches_played': '18',
  'matches_w': '13',
  'matches_l': '5',
  'points': '31'},
 {'position': '5',
  'team': 'GRUPO URETA TIZONA BURGOS',
  'matches_played': '18',
  'matches_w': '12',
  'matches_l': '6',
  'points': '30'},
 {'position': '6',
  'team': 'ICG FORÇA LLEIDA',
  'matches_played': '18',
  'matches_w': '12',
  'matches_l': '6',
  'points': '30'},
 {'position': '7',
  'team': 'UEMC REAL VALLADOLID BALONCESTO',
  'matches_played': '18',
  'matches_w': '11',
  'matches_l': '7',
  'points': '29'},
 {'position': '8'